# AI-Enhanced Qualitative Data Analysis

This notebook processes Excel workbooks containing qualitative data and uses AI models (BERTopic or Top2Vec) to automatically extract themes and topics for qualitative data analysis (QDA).

## Features
- Load multiple Excel sheets/tabs
- Extract qualitative text from specified columns
- Apply BERTopic for hierarchical theme extraction
- Alternative Top2Vec implementation
- Interactive visualizations
- Export results for QDA tools (Taguette, QualCoder, etc.)

## Setup
1. Place your Excel workbook in the `data/raw/` directory
2. Update the `excel_file` path below
3. Customize column names if needed

In [1]:
# Install required libraries if not already installed
# Uncomment the line below if running in an environment without these packages
# !pip3 install -r ../requirements.txt

In [2]:
import pandas as pd
import numpy as np
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')


# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

print("Libraries imported successfully!")

Libraries imported successfully!


In [3]:
# Step 1: Load the Excel workbook
# Replace 'your_workbook.xlsx' with the path to your Excel file
excel_file = '../data/raw/callcenterdata.xlsx'

# Load all sheets into a dictionary of DataFrames
try:
    sheets_dict = pd.read_excel(excel_file, sheet_name=None)
    print(f"✓ Successfully loaded workbook: {excel_file}")
    print(f"Loaded sheets: {list(sheets_dict.keys())}")
    
    # Display basic info about each sheet
    for sheet_name, df in sheets_dict.items():
        print(f"  - {sheet_name}: {df.shape[0]} rows, {df.shape[1]} columns")
        
except FileNotFoundError:
    print(f"✗ Error: File '{excel_file}' not found.")
    print("Please place your Excel file in the data/raw/ directory and update the path above.")
    sheets_dict = {}
except Exception as e:
    print(f"✗ Error loading Excel file: {e}")
    sheets_dict = {}

✓ Successfully loaded workbook: ../data/raw/callcenterdata.xlsx
Loaded sheets: ['COLA FONL Unique', 'General Inqury Unique', 'Unique Values', 'Contact Reason Details', 'Summary', 'Filtered Patty', 'Further anlaysis']
  - COLA FONL Unique: 2517 rows, 11 columns
  - General Inqury Unique: 1583 rows, 11 columns
  - Unique Values: 57974 rows, 4 columns
  - Contact Reason Details: 31879 rows, 11 columns
  - Summary: 247 rows, 4 columns
  - Filtered Patty: 249 rows, 10 columns
  - Further anlaysis: 8 rows, 16 columns


In [4]:
# Step 2: Extract qualitative text data
# Configure text column names to extract from
text_columns = ['Comment']  # Add your column names here

all_texts = []
text_sources = []  # Track which sheet each text came from

if sheets_dict:
    for sheet_name, df in sheets_dict.items():
        print(f"\nProcessing sheet: {sheet_name}")
        
        # Find text columns in this sheet
        available_text_cols = [col for col in text_columns if col in df.columns]
        
        if available_text_cols:
            print(f"  Found text columns: {available_text_cols}")
            
            # Extract text from each available column
            for col in available_text_cols:
                texts = df[col].dropna().astype(str).tolist()
                all_texts.extend(texts)
                text_sources.extend([f"{sheet_name}:{col}"] * len(texts))
                print(f"    - {col}: {len(texts)} texts extracted")
        else:
            print(f"  Warning: No text columns found in '{sheet_name}'. Available columns: {list(df.columns)}")
    
    print(f"\n✓ Total texts extracted: {len(all_texts)}")
    
    if all_texts:
        print(f"Sample text: {all_texts[0][:100]}...")
        
        # Save processed data
        processed_df = pd.DataFrame({
            'text': all_texts,
            'source': text_sources
        })
        processed_df.to_csv('../data/processed/extracted_texts.csv', index=False)
        print("✓ Processed texts saved to data/processed/extracted_texts.csv")
    else:
        print("✗ No texts were extracted. Please check your column names.")
else:
    print("No workbook loaded. Please check Step 1.")


Processing sheet: COLA FONL Unique
  Found text columns: ['Comment']
    - Comment: 2517 texts extracted

Processing sheet: General Inqury Unique
  Found text columns: ['Comment']
    - Comment: 1582 texts extracted

Processing sheet: Unique Values
  Found text columns: ['Comment']
    - Comment: 57944 texts extracted

Processing sheet: Contact Reason Details
  Found text columns: ['Comment']
    - Comment: 31865 texts extracted

Processing sheet: Summary

Processing sheet: Filtered Patty

Processing sheet: Further anlaysis

✓ Total texts extracted: 93908
Sample text:  	ADDRESS VERIFICATION FOR PERMIT FL-I-15562...
✓ Processed texts saved to data/processed/extracted_texts.csv


In [5]:
# Step 3: Apply BERTopic for theme extraction
# Select model to run ('bertopic' or 'top2vec') -- uses existing value if set
model_choice = globals().get('model_choice', 'bertopic')

if all_texts:
    print(f"Running {model_choice.upper()} analysis on {len(all_texts)} texts...")
    
    # Handle large datasets by sampling
    if len(all_texts) > 50000:
        print(f"Dataset is large ({len(all_texts)} texts). Sampling to 50,000 for analysis to avoid memory issues.")
        sample_indices = np.random.choice(len(all_texts), 50000, replace=False)
        analysis_texts = [all_texts[i] for i in sample_indices]
        analysis_sources = [text_sources[i] for i in sample_indices]
    else:
        analysis_texts = all_texts
        analysis_sources = text_sources
    
    if model_choice == 'bertopic':
        # Import additional libraries for custom models
        from umap import UMAP
        from hdbscan import HDBSCAN
        
        # Initialize BERTopic model with optimized settings for large datasets
        topic_model = BERTopic(
            language="english",
            calculate_probabilities=False,  # Set to False to reduce memory usage
            verbose=True,
            min_topic_size=5,  # Increased from 5 to reduce number of topics and memory
            nr_topics="auto",   # Auto-determine number of topics
            low_memory=True,  # Enable low memory mode for large datasets
            umap_model=UMAP(n_jobs=1),  # Disable multiprocessing in UMAP
            hdbscan_model=HDBSCAN(core_dist_n_jobs=1)  # Disable multiprocessing in HDBSCAN
        )
        
        # Fit the model
        topics, probabilities = topic_model.fit_transform(analysis_texts)
        
        # Get topic information
        theme_info = topic_model.get_topic_info()
        
        print(f"\n✓ BERTopic identified {len(theme_info)-1} topics (excluding outliers)")
        
    elif model_choice == 'top2vec':
        # Initialize Top2Vec model
        topic_model = Top2Vec(
            documents=analysis_texts,
            speed="learn",  # 'fast-learn', 'learn', 'deep-learn'
        )
        
        # Get topic information
        topic_sizes, topic_nums = topic_model.get_topic_sizes()
        topics_words = topic_model.get_topics()
        
        # Create theme_info DataFrame
        theme_info = pd.DataFrame({
            'Topic': topic_nums,
            'Count': topic_sizes,
            'Name': [', '.join(words[0][:10]) for words in topics_words],  # Top 10 words
            'Representation': [words[0][:10] for words in topics_words]
        })
        
        print(f"\n✓ Top2Vec identified {len(theme_info)} topics")
    
    # Display extracted themes
    display(theme_info.head(10))
    
else:
    print("No texts available for analysis. Please check Step 2.")
    theme_info = None

2026-01-28 14:57:17,899 - BERTopic - Embedding - Transforming documents to embeddings.


Running BERTOPIC analysis on 93908 texts...
Dataset is large (93908 texts). Sampling to 50,000 for analysis to avoid memory issues.


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Batches:   0%|          | 0/1563 [00:00<?, ?it/s]

2026-01-28 14:57:46,935 - BERTopic - Embedding - Completed ✓
2026-01-28 14:57:46,935 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2026-01-28 14:58:27,489 - BERTopic - Dimensionality - Completed ✓
2026-01-28 14:58:27,490 - BERTopic - Cluster - Start clustering the reduced embeddings
2026-01-28 14:58:28,068 - BERTopic - Cluster - Completed ✓
2026-01-28 14:58:28,068 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2026-01-28 14:58:28,525 - BERTopic - Representation - Completed ✓
2026-01-28 14:58:28,526 - BERTopic - Topic reduction - Reducing number of topics
2026-01-28 14:58:31,014 - BERTopic - Representation - Fine-tuning topics using representation models.
2026-01-28 14:58:31,483 - BERTopic - Representation - Completed ✓
2026-01-28 14:58:31,487 - BERTopic - Topic reduction - Reduced number of topics from 2256 to 1262



✓ BERTopic identified 1261 topics (excluding outliers)


,Topic,Count,Name,Representation,Representative_Docs
0,-1,8311,-1_x000d_and_product_alcohol,"[x000d, and, product, alcohol, is, that, be, i...",[Caller asked general questions about malt bev...
1,0,538,0_ttb_return_x000d_id,"[ttb, return, x000d, id, fedex, locate, 243410...","[TTB#23065001001026, TTB#25213001000382, TTB#2..."
2,1,515,1_hang_hangup_up_hung,"[hang, hangup, up, hung, action, na, hange, ha...","[hang up, hang up, Hang up]"
3,2,418,2_ds_labeling_questions_with,"[ds, labeling, questions, with, im, quesstions...","[im with ds labeling questions, im with ds lab..."
4,3,379,3_ds_dss_rakija_bam,"[ds, dss, rakija, bam, dis, ws, arak, canadian...","[DS, DS, DS]"
5,4,376,4_revision_allowable_revisions_10,"[revision, allowable, revisions, 10, list, net...","[Referred allowable revision# 3(d), Allowable ..."
6,5,369,5_wine_os_cooking_frutis,"[wine, os, cooking, frutis, bootcamp, 1wine, d...","[WINE, WINE, WINE]"
7,6,362,6_vm_asmw_surrender_applications,"[vm, asmw, surrender, applications, appellatio...","[vm, VM, VM]"
8,7,355,7_fonl_transferred_inquire_specific,"[fonl, transferred, inquire, specific, rejecte...","[transferred to FONL, transferred to FONL, tra..."
9,8,320,8_bring_compliance_into_how,"[bring, compliance, into, how, explained, labe...",[Explained how to bring the label into complia...


In [ ]:
# Step 4: Visualize topics
# Safely get variables that may not exist in the current kernel/session
theme_info = globals().get('theme_info', None)
model_choice = globals().get('model_choice', 'bertopic')
topic_model = globals().get('topic_model', None)

from IPython.display import display, HTML
import plotly.io as pio

# Prefer a non-blocking renderer for notebooks; HTML embedding fallback is safest
try:
    pio.renderers.default = 'notebook_connected'
except Exception:
    pass

def _display_fig(fig, title=None):
    """Embed a Plotly figure as HTML to avoid renderer blocking issues."""
    try:
        html = fig.to_html(full_html=False, include_plotlyjs='cdn')
        if title:
            display(HTML(f"<h4>{title}</h4>"))
        display(HTML(html))
    except Exception as e:
        print(f"Failed to render figure as HTML: {e}")

if theme_info is not None and model_choice == 'bertopic' and topic_model is not None:
    print("Generating visualizations...")
    
    # Topic visualization
    try:
        fig_topics = topic_model.visualize_topics()
        _display_fig(fig_topics, title='Topic Overview')
    except Exception as e:
        print(f"Topic visualization failed: {e}")
    
    # Topic hierarchy (if enough topics and not too many)
    try:
        if len(theme_info) > 3 and len(theme_info) < 200:
            fig_hierarchy = topic_model.visualize_hierarchy()
            _display_fig(fig_hierarchy, title='Topic Hierarchy')
        else:
            if len(theme_info) >= 200:
                print('Skipping hierarchy visualization: too many topics to render.')
    except Exception as e:
        print(f"Hierarchy visualization failed: {e}")
    
    # Topic distribution
    try:
        fig_barchart = topic_model.visualize_barchart(top_n_topics=10)
        _display_fig(fig_barchart, title='Top Topics Barchart')
    except Exception as e:
        print(f"Barchart visualization failed: {e}")
        
elif theme_info is not None and model_choice == 'top2vec':
    print("Top2Vec visualizations not implemented in this notebook.")
    print("Consider using BERTopic for richer visualizations.")
    
    # Simple bar chart for Top2Vec
    plt.figure(figsize=(12, 6))
    plt.bar(range(len(theme_info)), theme_info['Count'])
    plt.xlabel('Topic Number')
    plt.ylabel('Number of Documents')
    plt.title('Top2Vec Topic Distribution')
    plt.show()
    
else:
    if theme_info is None:
        print("No themes available for visualization. Run the analysis cells first.")
    elif model_choice == 'bertopic' and topic_model is None:
        print("BERTopic model object not found in the session. Re-run the BERTopic analysis cell.")
    else:
        print("No visualizations available for the current configuration.")

In [ ]:
# Step 5: Export themes for validation/import into QDA tool
if theme_info is not None:
    # Export to CSV
    output_file = f"../data/results/extracted_themes_{model_choice}.csv"
    theme_info.to_csv(output_file, index=False)
    print(f"✓ Themes exported to '{output_file}'")
    
    # Export detailed topic information (BERTopic only)
    if model_choice == 'bertopic':
        # Get topic representations
        topic_representations = {}
        for topic_id in theme_info['Topic'].unique():
            if topic_id != -1:  # Skip outlier topic
                words = topic_model.get_topic(topic_id)
                topic_representations[topic_id] = words
        
        # Create detailed export
        detailed_themes = []
        for topic_id, words in topic_representations.items():
            detailed_themes.append({
                'topic_id': topic_id,
                'topic_name': f"Topic_{topic_id}",
                'top_words': ', '.join([word for word, _ in words[:10]]),
                'word_scores': ', '.join([f"{word}:{score:.3f}" for word, score in words[:10]]),
                'document_count': theme_info[theme_info['Topic'] == topic_id]['Count'].iloc[0]
            })
        
        detailed_df = pd.DataFrame(detailed_themes)
        detailed_output = f"../data/results/detailed_themes_{model_choice}.csv"
        detailed_df.to_csv(detailed_output, index=False)
        print(f"✓ Detailed themes exported to '{detailed_output}'")
    
    # Export text-topic assignments for QDA validation
    if model_choice == 'bertopic':
        text_topics_df = pd.DataFrame({
            'text': analysis_texts,
            'source': analysis_sources,
            'topic': topics,
            'topic_probability': [None] * len(topics)  # Probabilities disabled for memory efficiency
        })
    else:
        # For Top2Vec, we'd need to get document-topic assignments
        text_topics_df = pd.DataFrame({
            'text': analysis_texts,
            'source': analysis_sources,
            'topic': ['Top2Vec_topic'] * len(analysis_texts)  # Placeholder
        })
    
    assignments_file = f"../data/results/text_topic_assignments_{model_choice}.csv"
    text_topics_df.to_csv(assignments_file, index=False)
    print(f"✓ Text-topic assignments exported to '{assignments_file}'")
    
    print("\n" + "="*50)
    print("EXPORT COMPLETE")
    print("="*50)
    print("Files ready for import into QDA tools like:")
    print("- Taguette (import CSV files)")
    print("- QualCoder (import CSV files)")
    print("- NVivo (import CSV files)")
    print("- ATLAS.ti (import CSV files)")
    
else:
    print("No themes to export.")

## Next Steps

1. **Validate AI-generated themes** in your QDA tool
2. **Refine topics** by merging, splitting, or renaming as needed
3. **Code additional texts** using the validated themes
4. **Export coded data** for further quantitative analysis

## Tips for Better Results

- **Preprocessing**: Clean your text data before analysis (remove noise, standardize formats)
- **Model Tuning**: Adjust `min_topic_size` in BERTopic for more/fewer topics
- **Language**: Set the correct language if your texts aren't in English
- **Scale**: For large datasets (>10k texts), consider running on Databricks Community Edition

## Troubleshooting

- **Memory issues**: Reduce `min_topic_size` or use Top2Vec instead
- **Poor topics**: Preprocess text better or try different model settings
- **No texts found**: Check column names in your Excel file